In [1]:
import os
from decimal import *
from copy import copy

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from scipy.stats import norm
from scipy.stats import multivariate_normal
import GPy

from preferentialBO.src.preferential_gp_regression import *

plt.rcParams.update(
    {"text.usetex": True, "font.family": "sans-serif", "font.sans-serif": ["Helvetica"]}
)
plt.rc("text.latex", preamble=r"\usepackage{bm} ")
plt.rcParams["figure.constrained_layout.use"] = False

plt.rcParams["font.size"] = 30
plt.rcParams["axes.titlesize"] = 30
plt.rcParams["lines.linewidth"] = 3.0
plt.rcParams["image.cmap"] = "plasma"
plt.rcParams["axes.labelpad"] = 0.2
plt.rcParams["axes.axisbelow"] = True

plt.rcParams["legend.fontsize"] = 30
plt.rcParams["legend.borderpad"] = 0.2  # border whitespace
plt.rcParams[
    "legend.labelspacing"
] = 0.2  # the vertical space between the legend entries
plt.rcParams["legend.handlelength"] = 1.0  # the length of the legend lines
plt.rcParams["legend.handleheight"] = 0.3  # the height of the legend handle
plt.rcParams[
    "legend.handletextpad"
] = 0.3  # the space between the legend line and legend text
plt.rcParams[
    "legend.borderaxespad"
] = 0.1  # the border between the axes and legend edge
plt.rcParams["legend.columnspacing"] = 1.0  # column separation
plt.rcParams["image.cmap"] = "tab20c"

In [2]:
from math import floor, log10


# Define function for string formatting of scientific notation
def sci_notation(num, decimal_digits=1, precision=None, exponent=None):
    """
    Returns a string representation of the scientific
    notation of the given number formatted for use with
    LaTeX or Mathtext, with specified number of significant
    decimal digits and precision (number of decimal digits
    to show). The exponent to be used can also be specified
    explicitly.
    """
    if exponent is None:
        exponent = int(floor(log10(abs(num))))
    coeff = round(num / float(10**exponent), decimal_digits)
    if precision is None:
        precision = decimal_digits

    return r"${0:.{2}f}\cdot10^{{{1:d}}}$".format(coeff, exponent, precision)

In [3]:
np.random.seed(0)

ls = 0.05
kernel = GPy.kern.RBF(input_dim=1, variance=1, lengthscale=ls)

candidate_size = 201
X = np.c_[np.linspace(0, 1, candidate_size)]
prior_mean = np.zeros(candidate_size)
prior_cov = kernel.K(X, X)
prior_chol = np.linalg.cholesky(prior_cov + 1e-8 * np.eye(candidate_size))
true_f = np.c_[prior_mean] + prior_chol.dot(np.c_[np.random.randn(candidate_size)])
# true_f = X
x_max_idx = np.argmax(true_f[100:]) + 100

plt.plot(np.ravel(X), true_f)
plt.xticks([], [])
plt.yticks([], [])
plt.xlim(0, 1)
plt.ylim(-2, 3.3)
# plt.savefig('../latent_func.pdf')
plt.show()
plt.close()

 /Users/takeno/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning:Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [4]:
# Problem dimension
dim = 50
seed = 0
input_dim = 1

# seed for reproducibility
np.random.seed(seed)

noise_std = 1e-2
x_bound = [np.zeros(input_dim), np.ones(input_dim)]

idx = np.random.choice(candidate_size, 2 * dim)
idx = idx.ravel().tolist()

X_init = X[idx].reshape(dim, 2 * input_dim)
y_init = true_f[idx]

y_init = y_init.reshape(dim, 2)

sort_ind = np.argsort(y_init, axis=1)

X_init_sort = np.take_along_axis(X_init, sort_ind, axis=1)[:, ::-1]
kernel = GPy.kern.RBF(input_dim=input_dim, lengthscale=ls, variance=1, ARD=True)

idx = np.array(idx).reshape(dim, 2 * input_dim)
idx = np.take_along_axis(idx, sort_ind, axis=1)[:, ::-1]
# print(idx)

In [5]:
burn_in = 2000
thinning = 10
np.random.seed(0)
PGP_Gibbs_acc = PreferentialGP_Gibbs(
    X_init_sort,
    kernel,
    kernel_bounds=x_bound,
    noise_std=noise_std,
    burn_in=burn_in,
    thinning=thinning,
    sample_size=10000,
)
PGP_Gibbs_acc.inference()

mean_acc = PGP_Gibbs_acc.mean(X)
upper_acc = PGP_Gibbs_acc.quantile(X, 0.975)
lower_acc = PGP_Gibbs_acc.quantile(X, 0.025)

In [6]:
i = 3
focus_x_idx = [idx[i, 0], idx[i, 1]]

A = np.c_[-1 * np.eye(dim), np.eye(dim)]
flatten_X = np.r_[X_init_sort[:, :input_dim], X_init_sort[:, input_dim:]]
X_test = X[focus_x_idx]

lower_f = PGP_Gibbs_acc.quantile(X_test, 0.001) - 0.5
upper_f = PGP_Gibbs_acc.quantile(X_test, 0.999) + 0.5

plot_f_fir = np.linspace(np.min(lower_f), np.max(upper_f), 51)
F1_fir, F2_fir = np.meshgrid(plot_f_fir, plot_f_fir)
F_fir = np.c_[F1_fir.ravel(), F2_fir.ravel()]

test_point_size = np.shape(X_test)[0]
transform_matrix = np.r_[
    np.c_[np.eye(test_point_size), np.zeros((test_point_size, dim))],
    np.c_[np.zeros((2 * dim, test_point_size)), A.T],
]

K_X_flattenX = kernel.K(np.r_[X_test, flatten_X], np.r_[X_test, flatten_X])

K_X_v = (
    transform_matrix.T
    @ (
        K_X_flattenX
        + noise_std**2 * np.diag(np.r_[np.zeros(test_point_size), np.ones(2 * dim)])
    )
    @ transform_matrix
)
Z = multivariate_normal.cdf(
    np.zeros(dim),
    np.zeros(dim),
    cov=K_X_v[test_point_size:, test_point_size:],
    abseps=1e-5,
    releps=1e-5,
)

tmp = K_X_v[test_point_size:, :test_point_size] @ np.linalg.inv(
    K_X_v[:test_point_size, :test_point_size]
)
conditional_cov = (
    K_X_v[test_point_size:, test_point_size:]
    - tmp @ K_X_v[test_point_size:, :test_point_size].T
)


def ESN_pdf(f):
    pdf = multivariate_normal.pdf(
        f, mean=np.zeros(2), cov=K_X_v[:test_point_size, :test_point_size]
    )
    conditional_mean = tmp @ f
    conditional_Z = multivariate_normal.cdf(
        np.zeros(dim),
        conditional_mean.ravel(),
        cov=conditional_cov,
        abseps=1e-4,
        releps=1e-4,
    )
    return pdf * conditional_Z / Z


ESN_pdf_list = []
for i in range(np.size(plot_f_fir) ** 2):
    ESN_pdf_list.append(ESN_pdf(F_fir[i]))
ESN_pdf_acc = np.array(ESN_pdf_list)

In [7]:
PGP_LP = PreferentialGP_Laplace(
    X_init_sort, kernel, kernel_bounds=x_bound, noise_std=noise_std
)
PGP_LP.inference()

mean_LP, var_LP = PGP_LP.predict(X)
mean_LP = mean_LP.ravel()
var_LP = var_LP.ravel()
std_LP = np.sqrt(var_LP)

In [8]:
PGP_EP = PreferentialGP_EP(
    X_init_sort, kernel, kernel_bounds=x_bound, noise_std=noise_std
)
PGP_EP.inference()

mean_EP, var_EP = PGP_EP.predict(X)
mean_EP = mean_EP.ravel()
var_EP = var_EP.ravel()
std_EP = np.sqrt(var_EP)

In [9]:
mean_LP_pdf, cov_LP_pdf = PGP_LP.predict(X[focus_x_idx], full_cov=True)
false_prob_LP = norm.cdf(
    0,
    loc=mean_LP_pdf[0] - mean_LP_pdf[1],
    scale=np.sqrt(cov_LP_pdf[1, 1] + cov_LP_pdf[0, 0] - 2 * cov_LP_pdf[0, 1]),
)

mean_EP_pdf, cov_EP_pdf = PGP_EP.predict(X[focus_x_idx], full_cov=True)
false_prob_EP = norm.cdf(
    0,
    loc=mean_EP_pdf[0] - mean_EP_pdf[1],
    scale=np.sqrt(cov_EP_pdf[1, 1] + cov_EP_pdf[0, 0] - 2 * cov_EP_pdf[0, 1]),
)

false_prob_exact_acc = PGP_Gibbs_acc.win_prob(X[focus_x_idx][0], X[focus_x_idx][1])

In [17]:
if not os.path.exists("Illustrative_example/"):
    os.makedirs("Illustrative_example/")

In [18]:
plt.rcParams["image.cmap"] = "viridis"
levels = [1e-3, 1e-2, 1e-1, 0.4]
y_adjst = 1.5


fig = plt.figure(figsize=(14, 5))
gs = GridSpec(1, 14)
ax1 = plt.subplot(gs[0, :9])
ax2 = plt.subplot(gs[0, 9:])

ax1.plot(
    np.ravel(X),
    true_f,
    color="black",
    linestyle="dashed",
    label="black-box utility function",
)
ax1.plot(X.ravel(), mean_acc, label="predictive mean")
ax1.fill_between(
    X.ravel(), lower_acc, upper_acc, color="lightblue", label=r"95 \% credible interval"
)

for i in range(len(focus_x_idx)):
    ax1.vlines(X[focus_x_idx[i]], -3, 3.5, linestyle="solid", color="red")
ax1.set_xticks([X[focus_x_idx[0]][0], X[focus_x_idx[1]][0]])
ax1.set_xticklabels([r"$x_w$", r"$x_l$"])
ax1.set_yticks([])
ax1.set_xlim(0, 1)
ax1.set_ylim(-3.0, 3.5)
# ax1.legend(loc="upper left", ncol=3, bbox_to_anchor=(0, 1.27))
ax1.set_title("MCMC")
# ax1.legend(loc="upper left")

# ax2.hist2d(sample_path[focus_x_idx[0], :], sample_path[focus_x_idx[1], :], bins=[50, 50], density=True)
# heatmap_exact = ax2.pcolor(F1_fir, F2_fir, ESN_pdf_acc.reshape(np.size(plot_f_fir), np.size(plot_f_fir)))
# fig.colorbar(heatmap_exact, ax=ax2, label=r"${\rm PDF}(f(x_w), f(x_l))$", ticks=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
heatmap_exact = ax2.contour(
    F1_fir,
    F2_fir,
    ESN_pdf_acc.reshape(np.size(plot_f_fir), np.size(plot_f_fir)),
    levels=levels,
)

fmt = {}
for l in levels:
    fmt[l] = str(l)
manual_location = [(1.0, -1), (1.1, -0.5), (1.3, -0.1), (1.4, 0.5)]
ax2.clabel(heatmap_exact, fontsize=18, colors="black", fmt=fmt, manual=manual_location)

ax2.plot(
    plot_f_fir,
    plot_f_fir,
    linestyle="dashed",
    color="lightgreen",
    label=r"$f(x_w) \! = \! f(x_l)$",
)
# ax2.legend(loc="lower right")
ax2.set_xticks([])
ax2.set_yticks([])

ax2.set_ylabel(r"$f(x_l)$")
ax2.set_xlabel(r"$f(x_w)$")
# ax2.set_title(r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\! = $ " + sci_notation(false_prob_exact_acc, 1, 1))
ax2.set_title(r"${\rm PDF}(f(x_w), f(x_l))$")
ax2.text(
    np.min(plot_f_fir) + 0.1,
    np.max(plot_f_fir) - y_adjst,
    r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\!$ "
    + "\n"
    + "$= {:.3f}$ ".format(false_prob_exact_acc),
)
# ax2.set_title(r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\! = {:.4f}$ ".format(false_prob_exact_acc))


handles, labels = ax1.get_legend_handles_labels()

plt.tight_layout()
plt.savefig("Illustrative_example/preferentialGP_MCMC_acc.pdf")
plt.show()
plt.close()

 /Users/takeno/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: UserWarning:Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [1]:
legend_fig = plt.figure(figsize=(14, 2))
legend_fig_ax = legend_fig.add_subplot(1, 1, 1)
legend_fig_ax.plot(
    [1], [1], color="black", linestyle="dashed", label="black-box utility function"
)
legend_fig_ax.plot([1], [1], label="predictive mean")
legend_fig_ax.fill_between(
    [1], [1], [1], color="lightblue", label=r"95 \% credible interval"
)
legend_fig_ax.plot(
    [1], [1], color="lightgreen", linestyle="dashed", label=r"$f(x_w) \! = \! f(x_l)$"
)
legend_fig_ax.axis("off")
legend_fig_ax.legend(loc="upper left", ncol=2)
plt.tight_layout()
plt.savefig("Illustrative_example/preferentialGP_legend.pdf")
plt.show()
plt.close()

NameError: name 'plt' is not defined

In [20]:
fig = plt.figure(figsize=(14, 5))
gs = GridSpec(1, 14)
ax4 = plt.subplot(gs[0, :9])
ax5 = plt.subplot(gs[0, 9:])
# ax6 = plt.subplot(gs[0, 3])

# PGP_LP.sample(sample_size=8)
# sample_path_LP = PGP_LP.evaluate_sample(X)

# for i in range(8):
#     ax4.plot(np.ravel(X), sample_path_LP[:,i], linestyle="dotted")

ax4.plot(
    np.ravel(X),
    true_f,
    color="black",
    linestyle="dashed",
    label="black-box utility function",
)
ax4.plot(X.ravel(), mean_LP, label="prediction")
ax4.fill_between(
    X.ravel(),
    mean_LP - 1.96 * std_LP,
    mean_LP + 1.96 * std_LP,
    color="lightblue",
    label="uncertainty",
)
for i in range(len(focus_x_idx)):
    ax4.vlines(X[focus_x_idx[i]], -3, 3.5, linestyle="solid", color="red")
ax4.set_xticks([X[focus_x_idx[0]][0], X[focus_x_idx[1]][0]])
ax4.set_xticklabels([r"$x_w$", r"$x_l$"])
ax4.set_yticks([])
# ax4.set_xticklabels([r"$x^{(1)}_{t+1}$"])
ax4.set_xlim(0, 1)
ax4.set_ylim(-3.0, 3.5)
ax4.set_title("Laplace approximation")
# ax4.legend(loc="upper left")

# ax4.plot(X.ravel(), mean_acc, label="predictive mean", color="red")
# ax4.fill_between(X.ravel(), lower_acc, upper_acc, color="pink", alpha=0.5, label=r"95 \% credible interval")

pdf = multivariate_normal.pdf(F_fir, mean=mean_LP_pdf.ravel(), cov=cov_LP_pdf)
heatmap_exact = ax5.contour(
    F1_fir, F2_fir, pdf.reshape(np.size(plot_f_fir), np.size(plot_f_fir)), levels=levels
)
ax5.plot(
    plot_f_fir,
    plot_f_fir,
    linestyle="dashed",
    color="lightgreen",
    label=r"$f(x_w) \! = \! f(x_l)$",
)
# ax5.legend(loc="lower right")
ax5.set_xticks([])
ax5.set_yticks([])

ax5.set_ylabel(r"$f(x_l)$")
ax5.set_xlabel(r"$f(x_w)$")
# ax5.set_title(r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\! = $ " + sci_notation(false_prob_exact_acc, 1, 1))
ax5.set_title(r"${\rm PDF}(f(x_w), f(x_l))$")
ax5.text(
    np.min(plot_f_fir) + 0.1,
    np.max(plot_f_fir) - y_adjst,
    r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\!$ "
    + "\n"
    + "$= {:.3f}$ ".format(false_prob_LP[0]),
)

plt.tight_layout()
plt.savefig("Illustrative_example/preferentialGP_LP.pdf")
plt.show()
plt.close()

 /Users/takeno/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning:Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [21]:
fig = plt.figure(figsize=(14, 5))
gs = GridSpec(1, 14)
ax7 = plt.subplot(gs[0, :9])
ax8 = plt.subplot(gs[0, 9:])
# ax9 = plt.subplot(gs[0, 3])


ax7.plot(
    np.ravel(X),
    true_f,
    color="black",
    linestyle="dashed",
    label="black-box utility function",
)
ax7.plot(X.ravel(), mean_EP, label="prediction")
ax7.fill_between(
    X.ravel(),
    mean_EP - 1.96 * std_EP,
    mean_EP + 1.96 * std_EP,
    color="lightblue",
    label="uncertainty",
)
for i in range(len(focus_x_idx)):
    ax7.vlines(X[focus_x_idx[i]], -3, 3.5, linestyle="solid", color="red")
ax7.set_xticks([X[focus_x_idx[0]][0], X[focus_x_idx[1]][0]])
ax7.set_xticklabels([r"$x_w$", r"$x_l$"])
ax7.set_yticks([])
# ax7.set_xticklabels([r"$x^{(1)}_{t+1}$"])
ax7.set_xlim(0, 1)
ax7.set_ylim(-3.0, 3.5)
ax7.set_title("Expectation propagation")
# ax7.legend(loc="upper left")

# ax7.plot(X.ravel(), mean_acc, label="predictive mean", color="red")
# ax7.fill_between(X.ravel(), lower_acc, upper_acc, color="pink", alpha=0.5, label=r"95 \% credible interval")

pdf = multivariate_normal.pdf(F_fir, mean=mean_EP_pdf.ravel(), cov=cov_EP_pdf)
heatmap_ep = ax8.contour(
    F1_fir, F2_fir, pdf.reshape(np.size(plot_f_fir), np.size(plot_f_fir)), levels=levels
)
# ax8.clabel(heatmap_ep)
ax8.plot(
    plot_f_fir,
    plot_f_fir,
    linestyle="dashed",
    color="lightgreen",
    label=r"$f(x_w) \! = \! f(x_l)$",
)
# ax8.legend(loc="lower right")
ax8.set_xticks([])
ax8.set_yticks([])

ax8.set_ylabel(r"$f(x_l)$")
ax8.set_xlabel(r"$f(x_w)$")
# ax8.set_title(r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\! = $ " + sci_notation(false_prob_exact_acc, 1, 1))
ax8.set_title(r"${\rm PDF}(f(x_w), f(x_l))$")
ax8.text(
    np.min(plot_f_fir) + 0.1,
    np.max(plot_f_fir) - y_adjst,
    r"$\Pr \bigl( f(x_w) \! \leq \! f(x_l) \bigr)\!$ "
    + "\n"
    + "$= {:.3f}$ ".format(false_prob_EP),
)

plt.tight_layout()
plt.savefig("Illustrative_example/preferentialGP_EP.pdf")
plt.show()
plt.close()

 /Users/takeno/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning:Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
